In [13]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [7]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

11501568/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 2s 785us/step - loss: 0.2991 - accuracy: 0.9149
Epoch 2/5
1875/1875 [==============================] - 1s 710us/step - loss: 0.1438 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 1s 732us/step - loss: 0.1059 - accuracy: 0.9683
Epoch 4/5
1875/1875 [==============================] - 1s 710us/step - loss: 0.0878 - accuracy: 0.9735
Epoch 5/5
313/313 [==============================] - 0s 567us/step - loss: 0.0689 - accuracy: 0.9788


[0.06892058998346329, 0.9787999987602234]

In [10]:
path = "./Dataset/"

df = pd.read_csv(path+"대파Model.csv")
df

NameError: name 'pd' is not defined

In [15]:
df.columns

Index(['날짜', '평균강수량', '최대강수량', '평균기온', '최고기온', '최저기온', '평균습도', '최저습도', '평균풍속',
       '최대풍속', '대파가격', '대파거래량', '대파생산량', '대파저장이월량', '대파수입량', '대파수입금액',
       '대파재배면적', '고급휘발유', '보통휘발유', '자동차용경유', '소비자물가지수', '농축수산물', '공업제품', '집세',
       '공공서비스', '개인서비스', '근원물가', '생활물가'],
      dtype='object')

In [ ]:
train_dataset = df.sample(frac=0.8,random_state=0)
test_dataset = df.drop(train_dataset.index)